# Deliverable 1. Retrieve Weather Data.
---
1. Create a folder called `Weather_Database` to save all the files related with this deliverable.

2. Save the `Weather_Database_starter_code.ipynb` starter code to the `Weather_Database` folder and rename it as `Weather_Database.ipynb`.

3. Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

4. Use the `citipy` module to get the nearest city for each latitude and longitude combination.

5. Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key; also, it's critical to avoid publishing your API key on your GitHub repository.

6. Retrieve the following information from the API call:

    * Latitude and longitude

    * Maximum temperature

    * Percent humidity

    * Percent cloudiness

    * Wind speed

    * Weather description (for example, clouds, fog, light rain, clear sky)

7. Add the weather data to a new DataFrame.

8. Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.
---

In [1]:
# Import initial libraries
import numpy as np
import pandas as pd
from citipy import citipy

## Use the `np.random.uniform` function to generate a new set of 2,000 random latitudes and 2,000 longitudes.

In [2]:
# Create a set of random latitude and longitude combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)

# Use the zip function to create an iterator of tuples containing the latitude and longitude combinations
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list
coordinates = list(lat_lngs)

In [4]:
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

83.08909995751708 77.83288555906267
-53.4014148109366 -122.78526085894964
27.42962578875118 -122.90099406253755
79.7571506162038 150.86200546501982
12.8665016069653 70.74366263721117
-86.42869736243215 151.33132271596753
-42.868443332367896 -99.63795802847933
-50.43892996063087 107.03752441531378
-84.64486539224846 14.083346727320901
86.84536079606917 -122.4393520409689
69.21369195297248 -146.21272931725048
48.7859453738879 73.92732726087843
-52.957729122341554 -30.539315518944676
35.374445926292054 -32.24626637661859
-72.01482604599866 157.0351889612736
42.794307197548164 0.49464800355542593
49.6876050715 -27.095319124417472
27.39376343477788 156.62452702709584
-85.96880794300132 106.8764029077119
-43.014125627543265 -10.852869455132605
-54.43857647806704 -51.252321563152066
67.93848572284256 -123.40550352874372
-38.50928645738687 -84.6906938680273
-76.46472162850296 -149.30663259578986
-34.995712033076565 6.4543482362958
73.39327468071346 -144.94889390038009
-48.66942214889913 -24.26

## Use the `citipy` module to get the nearest city for each latitude and longitude combination.

In [5]:
# Create an empty list for holding the cities
cities = []

# Use a for loop to identify nearest city for each latitude and longitude combination using the citipy module
for coordinate in coordinates:
    # Use the citipy module to identify the nearest city to the current coordinate
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

710

## Import your OpenWeatherMap's API key and assemble the API call URL as a string variable. Recall to edit the `config.py` file to add your API key.

In [6]:
# Import the requests library
import requests

# Import the time library
import time

# Import the datetime module from the datetime library
from datetime import datetime

# Import the OpenWeatherMap's API key
from config import weather_api_key

In [7]:
# Assemble the OpenWeatherMap's API call
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

## Retrieve the following information from the API call.

- The latitude and longitude
- The Max temperature
- The % humidity
- The % cloudiness
- The Wind speed
- The Weather description, i.e., cloudy, fog, light rain, clear sky, etc. 

In [8]:
# Create an empty list to hold weather data for each city
city_data = []

# Print a message to indicate that the data retrieval starts
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters and set them to 1
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data for each city
for i, city in enumerate(cities):
        
    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create an endpoint URL for each city
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the url, record, and set numbers
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        city_weather = requests.get(city_url).json()
        # Parse out the latitude, longitude, max temp, humidity, cloudiness, wind, country, and weather description
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_w_description = city_weather["weather"][0]["description"]    
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description": city_w_description})
        
    
    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

# Indicate that the data retrieval is complete 
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | dikson
Processing Record 2 of Set 1 | rikitea
Processing Record 3 of Set 1 | san quintin
Processing Record 4 of Set 1 | chokurdakh
Processing Record 5 of Set 1 | kavaratti
Processing Record 6 of Set 1 | bluff
Processing Record 7 of Set 1 | castro
Processing Record 8 of Set 1 | busselton
Processing Record 9 of Set 1 | bredasdorp
Processing Record 10 of Set 1 | tuktoyaktuk
Processing Record 11 of Set 1 | fairbanks
Processing Record 12 of Set 1 | abay
Processing Record 13 of Set 1 | chuy
Processing Record 14 of Set 1 | ribeira grande
Processing Record 15 of Set 1 | saint-gaudens
Processing Record 16 of Set 1 | lagoa
Processing Record 17 of Set 1 | hasaki
Processing Record 18 of Set 1 | albany
Processing Record 19 of Set 1 | jamestown
Processing Record 20 of Set 1 | ushuaia
Processing Record 21 of Set 1 | norman wells
Processing Record 22 of Set 1 | lebu
Processing Record 23 of Set 1 | mataura
Proces

Processing Record 32 of Set 4 | maputo
Processing Record 33 of Set 4 | pisz
Processing Record 34 of Set 4 | fortuna
Processing Record 35 of Set 4 | kollam
Processing Record 36 of Set 4 | muros
Processing Record 37 of Set 4 | tsihombe
City not found. Skipping...
Processing Record 38 of Set 4 | dostpur
Processing Record 39 of Set 4 | airai
Processing Record 40 of Set 4 | arlit
Processing Record 41 of Set 4 | saint anthony
Processing Record 42 of Set 4 | waipawa
Processing Record 43 of Set 4 | tura
Processing Record 44 of Set 4 | punta arenas
Processing Record 45 of Set 4 | safaga
Processing Record 46 of Set 4 | sinnamary
Processing Record 47 of Set 4 | saint-philippe
Processing Record 48 of Set 4 | svetlogorsk
Processing Record 49 of Set 4 | iqaluit
Processing Record 50 of Set 4 | havre-saint-pierre
Processing Record 1 of Set 5 | mitsamiouli
Processing Record 2 of Set 5 | vikindu
Processing Record 3 of Set 5 | grindavik
Processing Record 4 of Set 5 | shenjiamen
Processing Record 5 of Set

Processing Record 22 of Set 8 | grants
Processing Record 23 of Set 8 | nemuro
Processing Record 24 of Set 8 | takoradi
Processing Record 25 of Set 8 | zhob
Processing Record 26 of Set 8 | axim
Processing Record 27 of Set 8 | narsaq
Processing Record 28 of Set 8 | andarab
City not found. Skipping...
Processing Record 29 of Set 8 | bentiu
Processing Record 30 of Set 8 | olafsvik
Processing Record 31 of Set 8 | dwarka
Processing Record 32 of Set 8 | coihaique
Processing Record 33 of Set 8 | nelson bay
Processing Record 34 of Set 8 | warqla
City not found. Skipping...
Processing Record 35 of Set 8 | tiksi
Processing Record 36 of Set 8 | wheatley
Processing Record 37 of Set 8 | disna
Processing Record 38 of Set 8 | kikwit
Processing Record 39 of Set 8 | soyaniquilpan
Processing Record 40 of Set 8 | ouesso
Processing Record 41 of Set 8 | haibowan
City not found. Skipping...
Processing Record 42 of Set 8 | saint-prosper
Processing Record 43 of Set 8 | tibati
Processing Record 44 of Set 8 | ni

Processing Record 11 of Set 12 | louisbourg
City not found. Skipping...
Processing Record 12 of Set 12 | jalu
Processing Record 13 of Set 12 | huarmey
Processing Record 14 of Set 12 | kimbe
Processing Record 15 of Set 12 | goma
Processing Record 16 of Set 12 | ndele
Processing Record 17 of Set 12 | kondrovo
Processing Record 18 of Set 12 | safwah
City not found. Skipping...
Processing Record 19 of Set 12 | mirnyy
Processing Record 20 of Set 12 | ixtapa
Processing Record 21 of Set 12 | te anau
Processing Record 22 of Set 12 | standish
Processing Record 23 of Set 12 | chapada dos guimaraes
Processing Record 24 of Set 12 | buriti
Processing Record 25 of Set 12 | quatre cocos
Processing Record 26 of Set 12 | tornio
Processing Record 27 of Set 12 | ruatoria
City not found. Skipping...
Processing Record 28 of Set 12 | kargil
Processing Record 29 of Set 12 | abu road
Processing Record 30 of Set 12 | hudiksvall
Processing Record 31 of Set 12 | jining
Processing Record 32 of Set 12 | iskateley


In [9]:
# Print the length of the city_data list to verify how many cities you have
len(city_data)

641

## Add the weather data to a new DataFrame.

In [10]:
# Use the city_data list to create a new pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

In [11]:
# Display sample data
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Dikson,73.5069,80.5464,-4.92,96,100,7.94,RU,2023-03-04 18:53:25,overcast clouds
1,Rikitea,-23.1203,-134.9692,79.99,75,100,15.99,PF,2023-03-04 18:53:25,overcast clouds
2,San Quintin,30.4833,-115.9500,58.50,68,0,15.17,MX,2023-03-04 18:53:26,clear sky
3,Chokurdakh,70.6333,147.9167,-34.71,96,14,6.46,RU,2023-03-04 18:53:26,few clouds
4,Kavaratti,10.5669,72.6420,82.24,58,0,4.27,IN,2023-03-04 18:53:27,clear sky
5,Bluff,-46.6000,168.3333,52.95,64,76,7.49,NZ,2023-03-04 18:53:27,broken clouds
6,Castro,-24.7911,-50.0119,74.14,80,12,8.68,BR,2023-03-04 18:53:28,few clouds
7,Busselton,-33.6500,115.3333,60.49,74,57,9.44,AU,2023-03-04 18:50:17,broken clouds
8,Bredasdorp,-34.5322,20.0403,67.12,88,88,7.63,ZA,2023-03-04 18:53:28,light rain
9,Tuktoyaktuk,69.4541,-133.0374,-34.60,74,0,3.44,CA,2023-03-04 18:53:29,clear sky


In [12]:
# Display the DataFrame's column names using the columns Pandas function
city_data_df.columns

Index(['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date', 'Current Description'],
      dtype='object')

In [13]:
# Create a list to reorder the column names as follows:
# "City", "Country", "Lat", "Lng", "Max Temp", "Humidity",  "Cloudiness", "Wind Speed",  "Current Description"
new_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed",  "Current Description"]

# Recreate the DataFrame by using the new column order
city_data_df = city_data_df[new_order]

# Display sample data
city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Dikson,RU,73.5069,80.5464,-4.92,96,100,7.94,overcast clouds
1,Rikitea,PF,-23.1203,-134.9692,79.99,75,100,15.99,overcast clouds
2,San Quintin,MX,30.4833,-115.9500,58.50,68,0,15.17,clear sky
3,Chokurdakh,RU,70.6333,147.9167,-34.71,96,14,6.46,few clouds
4,Kavaratti,IN,10.5669,72.6420,82.24,58,0,4.27,clear sky


In [14]:
# Display the data types of each column by using the dtypes Pandas function
city_data_df.dtypes

City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
dtype: object

## Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [17]:
# Set the output file name
output_data_file = "WeatherPy_Database.csv"

# Export the city_data DataFrame into a CSV file
city_data_df.to_csv(output_data_file, index_label="City_ID")